# Azure AI Agents with Model Context Protocol (MCP) Support - Python

या नोटबुकमध्ये Python मध्ये Model Context Protocol (MCP) साधनांसह Azure AI Agents कसे वापरायचे ते दाखवले आहे. हे दाखवते की बाह्य MCP सर्व्हर्स (उदाहरणार्थ Microsoft Learn) चा उपयोग करून, कीलेस प्रमाणीकरणासह प्रगत क्षमता मिळवण्यासाठी बुद्धिमान एजंट कसा तयार करायचा.


## आवश्यक Python पॅकेजेस स्थापित करा

सर्वप्रथम, आपल्याला आवश्यक Python पॅकेजेस स्थापित करावे लागतील:
- **azure-ai-projects**: मुख्य Azure AI Projects SDK
- **azure-ai-agents**: एजंट तयार करण्यासाठी आणि व्यवस्थापित करण्यासाठी Azure AI Agents SDK
- **azure-identity**: DefaultAzureCredential वापरून कीशिवाय प्रमाणीकरण प्रदान करते
- **mcp**: Python साठी Model Context Protocol ची अंमलबजावणी


## कीलेस ऑथेंटिकेशनचे फायदे

ही नोटबुक **कीलेस ऑथेंटिकेशन** प्रदर्शित करते, ज्यामुळे अनेक फायदे मिळतात:
- ✅ **API कीज व्यवस्थापित करण्याची गरज नाही** - Azure ओळख-आधारित ऑथेंटिकेशन वापरते
- ✅ **सुरक्षेत वाढ** - कोड किंवा कॉन्फिगरेशन फाइल्समध्ये कोणतेही गुपित साठवले जात नाही
- ✅ **स्वयंचलित क्रेडेन्शियल रोटेशन** - Azure क्रेडेन्शियल जीवनचक्र व्यवस्थापन हाताळते
- ✅ **रोल-आधारित प्रवेश नियंत्रण** - सूक्ष्म परवानग्यांसाठी Azure RBAC वापरते
- ✅ **मल्टी-एन्व्हायर्नमेंट समर्थन** - विकास आणि उत्पादनामध्ये सहजतेने कार्य करते

`DefaultAzureCredential` आपोआप सर्वोत्तम उपलब्ध क्रेडेन्शियल स्रोत निवडते:
1. **मॅनेज्ड आयडेंटिटी** (Azure मध्ये चालू असताना)
2. **Azure CLI** क्रेडेन्शियल्स (स्थानिक विकासादरम्यान)
3. **Visual Studio** क्रेडेन्शियल्स
4. **एन्व्हायर्नमेंट व्हेरिएबल्स** (जर कॉन्फिगर केले असतील)
5. **इंटरॅक्टिव्ह ब्राउझर** ऑथेंटिकेशन (शेवटचा पर्याय म्हणून)


## कीलेस ऑथेंटिकेशन सेटअप

**कीलेस ऑथेंटिकेशनसाठी पूर्वअट:**

### स्थानिक विकासासाठी:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Azure वातावरणासाठी:
- आपल्या Azure संसाधनावर **System-assigned Managed Identity** सक्षम करा
- व्यवस्थापित ओळखीसाठी योग्य **RBAC भूमिका** नियुक्त करा:
  - Azure OpenAI प्रवेशासाठी `Cognitive Services OpenAI User`
  - Azure AI प्रकल्पांच्या प्रवेशासाठी `AI Developer`

### पर्यावरणीय चल (ऐच्छिक):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**कुठल्याही API की किंवा कनेक्शन स्ट्रिंगची गरज नाही!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## आवश्यक लायब्ररी आयात करा

आवश्यक Python मॉड्यूल्स आयात करा:
- **os, time**: पर्यावरणीय व्हेरिएबल्स आणि विलंबासाठी मानक Python लायब्ररी
- **AIProjectClient**: Azure AI Projects साठी मुख्य क्लायंट
- **DefaultAzureCredential**: Azure सेवांसाठी कीशिवाय प्रमाणीकरण
- **MCP-संबंधित वर्ग**: MCP टूल्स तयार करण्यासाठी, व्यवस्थापित करण्यासाठी आणि मंजुरी हाताळण्यासाठी


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## MCP सर्व्हर सेटिंग्ज कॉन्फिगर करा

पर्यावरणीय व्हेरिएबल्ससह MCP सर्व्हर कॉन्फिगरेशन सेट करा, ज्यामध्ये डिफॉल्ट पर्याय आहेत:
- **MCP_SERVER_URL**: MCP सर्व्हरचा URL (डिफॉल्ट Microsoft Learn API)
- **MCP_SERVER_LABEL**: MCP सर्व्हर ओळखण्यासाठी लेबल (डिफॉल्ट "mslearn")

ही पद्धत विविध वातावरणांमध्ये लवचिक कॉन्फिगरेशनसाठी अनुमती देते.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Azure AI प्रकल्प क्लायंट तयार करा (कीलेस प्रमाणीकरण)

**कीलेस प्रमाणीकरण** वापरून Azure AI प्रकल्प क्लायंट प्रारंभ करा:
- **endpoint**: Azure AI Foundry प्रकल्पाचा endpoint URL
- **credential**: सुरक्षित, कीलेस प्रमाणीकरणासाठी `DefaultAzureCredential()` वापरते
- **API की आवश्यक नाहीत**: सर्वोत्तम उपलब्ध credential आपोआप शोधते आणि वापरते

**प्रमाणीकरण प्रक्रिया:**
1. व्यवस्थापित ओळख (Azure वातावरणात) तपासते
2. Azure CLI credentials (स्थानिक विकासासाठी) वापरण्यासाठी fallback करते
3. आवश्यकतेनुसार इतर उपलब्ध credential स्रोत वापरते

ही पद्धत तुमच्या कोडमध्ये API की किंवा कनेक्शन स्ट्रिंग्स व्यवस्थापित करण्याची गरज दूर करते.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## MCP साधन परिभाषा तयार करा

Microsoft Learn MCP सर्व्हरशी जोडणारे MCP साधन तयार करा:
- **server_label**: MCP सर्व्हरसाठी ओळखकर्ता
- **server_url**: MCP सर्व्हरचा URL एंडपॉइंट
- **allowed_tools**: कोणते साधने वापरता येतील यावर मर्यादा घालण्यासाठी पर्यायी यादी (रिकामी यादी सर्व साधनांना परवानगी देते)

हे साधन एजंटला Microsoft Learn दस्तऐवज आणि संसाधनांमध्ये प्रवेश करण्यास सक्षम करेल.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## एजंट तयार करा आणि संभाषण चालवा (कीलेस वर्कफ्लो)

ही सविस्तर विभाग **कीलेस एजंट वर्कफ्लो** पूर्णपणे समजावून सांगतो:

1. **AI एजंट तयार करा**: GPT-4.1 नॅनो मॉडेल आणि MCP साधनांसह एजंट सेट अप करा
2. **थ्रेड तयार करा**: संवादासाठी एक संभाषण थ्रेड तयार करा
3. **संदेश पाठवा**: एजंटला Azure OpenAI आणि OpenAI मधील फरक विचारून पहा
4. **साधन मंजुरी हाताळा**: आवश्यक असल्यास MCP साधन कॉल्स आपोआप मंजूर करा
5. **अंमलबजावणीचे निरीक्षण करा**: एजंटची प्रगती ट्रॅक करा आणि आवश्यक ती कृती हाताळा
6. **परिणाम दर्शवा**: संभाषण आणि साधन वापराचे तपशील दाखवा

**कीलेस वैशिष्ट्ये:**
- ✅ **हार्डकोडेड गुपिते नाहीत** - सर्व प्रमाणीकरण Azure ओळखीद्वारे हाताळले जाते
- ✅ **डिफॉल्टनुसार सुरक्षित** - भूमिका-आधारित प्रवेश नियंत्रणाचा वापर करते
- ✅ **सोपे उपयोजन** - कोणत्याही क्रेडेन्शियल व्यवस्थापनाची आवश्यकता नाही
- ✅ **ऑडिटसाठी अनुकूल** - सर्व प्रवेश Azure ओळखीद्वारे ट्रॅक केला जातो

एजंट MCP साधनांचा वापर करून Microsoft Learn संसाधनांमध्ये पूर्ण सुरक्षा आणि कोणत्याही API की व्यवस्थापनाशिवाय प्रवेश करेल.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात ठेवा की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून उद्भवलेल्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
